Goal: Automated table extraction for knowledge graph development. [Camelot](https://camelot-py.readthedocs.io/en/master/) provides tools to extract tables from pdf's to pandas data frames. The pdf is "Appendix C" that describes the data format for the form “OPNAV 4790/2K”. A useful [medium article](https://medium.com/@luchensf/retrieve-table-contents-from-pdf-df514b779d07) on extracting tables using Camelot.

## Install camelot from conda forge
```bash
mamba install -c conda-forge camelot-py
```

### Note:
Installation using mamba from the condaforge channel installs the ghostscript gs excutable dependency in the environments bin directory which probably won't be in your path. To fix this:

```bash
export PATH=/Users/cvardema/mambaforge/envs/pdfmunge/bin:$PATH
```

Additionally and annoyingly, conda installs the python [ghostscript](https://pypi.org/project/ghostscript/) package in the user site packages directory (.local) which may not be in the python path. Here I used sys.path.insert to insert the .local/lib/python3.10/site-packages into the correct python path.

Both of these paths (executable and module) must be set correctly or camelot will fail with irritating module not found error messages.


In [1]:
!export PATH=/Users/ccunnin8/mambaforge/envs/pdfmunge/bin:$PATH
import sys
sys.path.insert(0, "/Users/ccunnin8/.local/lib/python3.10/site-packages")
print(sys.path)

['/Users/ccunnin8/.local/lib/python3.10/site-packages', '/home/ccunnin8/git/decoder-ring', '/home/ccunnin8/mambaforge/envs/pdfmunge/lib/python310.zip', '/home/ccunnin8/mambaforge/envs/pdfmunge/lib/python3.10', '/home/ccunnin8/mambaforge/envs/pdfmunge/lib/python3.10/lib-dynload', '', '/home/ccunnin8/.local/lib/python3.10/site-packages', '/home/ccunnin8/mambaforge/envs/pdfmunge/lib/python3.10/site-packages']


In [2]:
import camelot
from camelot import utils
from pathlib import Path

In [3]:
import ghostscript

In [4]:
datapath = Path('./data')
pdf_table_reader = camelot.read_pdf('./data/JFMM-VI-19-2K.pdf',pages='all')

In [5]:
print("Number of Tables detected: ", pdf_table_reader.n)
print(pdf_table_reader[0].parsing_report)

Number of Tables detected:  47
{'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 3}


- Pandas df index 0: When Discovered Codes
- Pandas df index 1: Status Codes
- Pandas df index 2: Cause Codes
- Pandas df index 3-4: Deferral Codes
- Pandas df index 5: Safety Hazard Codes
- Pandas df index 6: Alteration Type Codes
- Pandas df index 7: Rank or Rate Codes
- Pandas df index 8: Priority Codes
- Pandas df index 9-10: Type Availiability Codes/Usage of Type Availiability Codes
- Pandas df index 11-12: Action To be Taken Codes
- Pandas df index 13-18: Action Taken Codes Part 1 (Verify on HTTPS://OARS.NSLC.NAVY.MIL/OARS/DOCS/REF/INDEX.HTML)
- Pandas df index 19: When Discovered Codes
- Pandas df index 20: Status Codes
- Pandas df index 21: Cause Codes
- Pandas df index 22: Safety Hazard Codes
- Pandas df index 23-26: Action Taken Codes Part 2

- Pandas df index 27-47: Junk





In [6]:
pdf_table_reader [4].df

,0,1
0,Code,Deferral Reason
1,5,Inadequate School Practical Training
2,6,Lack of Facilities or Capabilities
3,7,Not Authorized for Ship’s Force or Unit Accomp...
4,8,For Ship’s Force or Unit Overhaul of Availabil...
5,9,Lack of Technical Documentation
6,0,Other - or Not Applicable (explain in block 35)


In [7]:
import pandas as pd
#when discovered
whenDisc = pdf_table_reader[0].df
whenDisc.to_csv('tables/whenDiscoveredCode.csv')
#status
status = pdf_table_reader[1].df
status.to_csv('tables/statusCode.csv')
#cause
cause = pdf_table_reader[2].df
cause.to_csv('tables/causeCode.csv')
#deferral
table_c3 = pdf_table_reader[3].df
table_c41 = pdf_table_reader[4].df
table_c42 = table_c41.drop(table_c41.index[0])
df = pd.concat([table_c3,table_c42])
df.to_csv('tables/deferralCode.csv')
#safety hazard
safety = pdf_table_reader[5].df
safety.to_csv('tables/safetyHazardCode.csv')
#alteration type
alteration = pdf_table_reader[6].df
alteration.to_csv('tables/alterationCode.csv')
#rank or rate
rate = pdf_table_reader[7].df
rate.to_csv('tables/rateCode.csv')
#priority
priority = pdf_table_reader[8].df
priority.to_csv('tables/priorityCode.csv')
#type availability
type = pdf_table_reader[9].df
type.to_csv('tables/typeAvailabilityCode.csv')
#type availability scenarios
scenario = pdf_table_reader[10].df
scenario.to_csv('tables/typeAvailabilityScenarioCode.csv')
#action to be taken
table_c11 = pdf_table_reader[11].df
table_c121 = pdf_table_reader[12].df
table_c122 = table_c121.drop(table_c121.index[0])
df = pd.concat([table_c11,table_c122])
df.to_csv('tables/actionToBeTakenCode.csv')
#action taken 1
action1 = pdf_table_reader[13].df

action2 = pdf_table_reader[14].df
action21 = action2.iloc[:5:]
action22 = action2.iloc[5:,:]
action21.to_csv('tables/actionTakenCodes/at3_2ndCharacter.csv')

action3 = pdf_table_reader[15].df
action31 = action3.iloc[:5:]
action32 = action3.iloc[5:,:]
action31.to_csv('tables/actionTakenCodes/at7_2ndCharacter.csv')

action4 = pdf_table_reader[16].df
action41 = action4.iloc[:11:]
action41.loc[len(df.index)] = ['','A', 'FOTE, multimode heavy duty MQJs utilized']
action41.to_csv('tables/actionTakenCodes/at9_2ndCharacter.csv')
action42 = action4.iloc[11:,:]

action5 = pdf_table_reader[17].df
action51 = action5.drop(action5.index[0])
action6 = pdf_table_reader[18].df

firstNumber = pd.concat([action1,action22,action32,action42,action51])
firstNumber.to_csv('tables/actionTakenCodes/actionTakenPrimaryCode.csv')



In [8]:
import pandas as pd
table_c1 = pdf_table_reader[3].df
for index in range(3,4):
    table_c1.merge(pdf_table_reader[index].df)
table_c1.to_csv('merge1.csv')

In [9]:

for index in range(0,14):
    file_name = "./data/2k_datastruct_" + str(index) + ".csv"
    pdf_table_reader[index].df.to_csv(file_name)

In [10]:
#Table Cleanup for KG Building
from pathlib import PurePath, Path
#actionTakenCode
p = PurePath('./tables/actionTakenCodes/actionTakenPrimaryCode.csv')
df = pd.read_csv(p)
pd.set_option('display.max_columns', None)
df.columns = ['0','Code','Description','1']
df = df.iloc[pd.RangeIndex(len(df)).drop(10)]
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
display(df)
df.to_csv('tables/actionTakenCodes/actionTakenPrimaryCode.csv')
#actionToBeTakenCode
p = PurePath('./tables/actionToBeTakenCode.csv')
df = pd.read_csv(p)
pd.set_option('display.max_columns', None)
df.columns = df.iloc[0]
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
df = df.iloc[pd.RangeIndex(len(df)).drop(28)]
df.at[27,'Code']= 8
df.at[28,'Code']= 9
display(df)
df.to_csv('tables/actionToBeTakenCode.csv')
#alterationCode
p = PurePath('./tables/alterationCode.csv')
df = pd.read_csv(p)
pd.set_option('display.max_columns', None)
df.columns = ['0','Code','Description']
df.to_csv('tables/alterationCode.csv')
display(df)
#causeCode
p = PurePath('./tables/causeCode.csv')
df = pd.read_csv(p)
pd.set_option('display.max_columns', None)
df.columns = df.iloc[0]
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
display(df)
df.to_csv('tables/causeCode.csv')
#deferralCode
p = PurePath('./tables/deferralCode.csv')
df = pd.read_csv(p)
pd.set_option('display.max_columns', None)
df.columns = ['0','Code','Description']
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
display(df)
df.to_csv("tables/deferralCode.csv")
#priorityCode
p = PurePath('./tables/priorityCode.csv')
df = pd.read_csv(p)
pd.set_option('display.max_columns', None)
df.columns = ['0','Code','Description']
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
display(df)
df.to_csv("tables/priorityCode.csv")
#rateCode
p = PurePath('./tables/rateCode.csv')
df = pd.read_csv(p)
pd.set_option('display.max_columns', None)
df.columns = ['0','Description','Code']
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
display(df)
df.to_csv("tables/rateCode.csv")
#safetyHazardCode
p = PurePath('./tables/safetyHazardCode.csv')
df = pd.read_csv(p)
pd.set_option('display.max_columns', None)
df.columns = ['0','Code','Description']
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
df = df.iloc[pd.RangeIndex(len(df)).drop(6)]
display(df)
df.to_csv("tables/safetyHazardCode.csv")
#statusCode
p = PurePath('./tables/statusCode.csv')
df = pd.read_csv(p)
pd.set_option('display.max_columns', None)
df.columns = ['0','Code','Description']
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
display(df)
df.to_csv("tables/statusCode.csv")
#typeAvailabilityCode
p = PurePath('./tables/typeAvailabilityCode.csv')
df = pd.read_csv(p)
pd.set_option('display.max_columns', None)
df.columns = ['0','Code','Description']
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
display(df)
df.to_csv("tables/typeAvailabilityCode.csv")
#whenDiscoveredCode
p = PurePath('./tables/whenDiscoveredCode.csv')
df = pd.read_csv(p)
pd.set_option('display.max_columns', None)
df.columns = ['0','Code','Description']
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]
display(df)
df.to_csv("tables/whenDiscoveredCode.csv")

,0,Code,Description,1
2,2,1,Maintenance Action Completed; Parts Drawn from...,NaN
3,3,2,Maintenance Action Completed; Required Parts N...,NaN
4,4,3,Maintenance Action Completed; No Parts Required,NaN
5,5,4,"Canceled (When this code is used, the deferral...",NaN
6,6,7,Maintenance Action Completed; 2-M (Miniature a...,NaN
7,5,8,Periodic Time Meter or Cycle Counter reporting...,NaN
8,6,9,Maintenance Action Completed; 3-M Fiber Optic...,NaN
9,11,0,None of the Above,NaN
11,2,5A,Partially Completed Alteration,NaN
12,3,5B,Fully Completed Alteration,NaN


,0,Code,Description
1,1,1,Depot (shipyard or ship repair facility) Accom...
2,2,1A,Depot Assisted by Ship’s Force or Unit Personnel
3,3,1S,Ship to Shop
4,4,1M,Accomplish with Modification
5,5,2,"IMA (tender or repair ship, etc.) Accomplish"
6,6,2A,IMA Assisted by Ship’s Force or Unit Personnel
7,7,2S,Ship to Shop
8,8,2M,Accomplish with Modification
9,9,3,Fleet Technical Support. TYCOM Support Unit (...
10,10,3A,TYCOM Support Un Assisted by Ship’s Force or U...


,0,Code,Description
0,0,SA,Ship Alteration
1,1,OA,Ordnance Alteration
2,2,BA,Boat Alteration
3,3,FC,Field Change
4,4,MA,Machinery Alteration
5,5,SI,SYSCOM Command Instruction
6,6,EC,Engineering Change
7,7,HI,Habitability
8,8,TY,TYCOM Direction
9,9,TD,Technical Directive


,0,Code,Description
1,1,1,Abnormal Environment
2,2,2,Manufacturer or Installation Defects
3,3,3,Lack of Knowledge or Skill
4,4,4,Communication Problem
5,5,5,Inadequate Instruction or Procedure
6,6,6,Inadequate Design
7,7,7,Normal Wear and Tear
8,8,0,Other or No Malfunction


,0,Code,Description
1,1,1,"Due to Ship’s Force, Unit’s Work Backlog or Op..."
2,2,2,Lack of Material
3,3,3,No Formal Training on this Equipment
4,4,4,Formal Training Inadequate for this Equipment
5,1,5,Inadequate School Practical Training
6,2,6,Lack of Facilities or Capabilities
7,3,7,Not Authorized for Ship’s Force or Unit Accomp...
8,4,8,For Ship’s Force or Unit Overhaul of Availabil...
9,5,9,Lack of Technical Documentation
10,6,0,Other - or Not Applicable (explain in block 35)


...

,0,Description,Code
2,2,Officers,OFF
3,3,ET1,ET1
4,4,Civilian,CIV
5,5,GMG2,GMG2
6,6,FTGSN,FTGN
7,7,FN,FN


,0,Code,Description
2,2,1,Critical Safety or Health Deficiency-Correct I...
3,3,2,Serious Safety or Health Deficiency-Suspension...
4,4,3,Moderate Safety or Health Deficiency-Waiver of...
5,5,4,Minor Safety or Health Deficiency
6,6,5,Negligible Safety or Health Deficiency
7,7,0,Maintenance Action is Not Safety Related


,0,Code,Description
1,1,1,Operational
2,2,2,Non-Operational
3,3,3,Reduced Capability
4,4,0,Not Applicable (use if reporting printing serv...


,0,Code,Description
2,2,1,Depot (shipyard or ship repair facility)
3,3,2,"Intermediate Maintenance Activity (tender, rep..."
4,4,3,Fleet Technical Support. TYCOM Support Unit (...
5,5,4,"Ship’s Force or Unit (Originating Work Center,..."
6,6,0,Not Applicable
7,7,U,(Mission Degrading) (entered on 2K) Used by IN...


,0,Code,Description
1,1,1,Lighting Off or Starting
2,2,2,Normal Operation
3,3,3,During Operability Test
4,4,4,During Inspection
5,5,5,Shifting Operational Modes
6,6,6,During PMS
7,7,7,Securing
8,8,8,During AEC (Assessment of Equipment) Program
9,9,9,"No Failure, PMS Accomplishment Only"
10,10,0,Not Applicable (use when reporting printing se...
